# CS202: Compiler Construction

## In-class Exercises, Week of 04/24/2023

----

# Lexical and Dynamic Scope

## Question 1

What is the output of this program under *lexical scope*?

In [ ]:
z = 1
def f():
    z = 3
    def g():
        print(z)
    return g

z = 5
f()()

The answer is 3, becase we use the environment when 'g' was defined, not when it was called. 

## Question 2

What would the output be under *dynamic scope*?

The answer would be '5', because we used the environment that existed when 'g' was called, in which z=5.

## Question 3

What is the difference between lexical and dynamic scope?

When evaluating a function body when the function is called:
- Lexical scope uses the environment that existed when the function was defined
- Dynamic scope uses the environment that existed when the function was called

When writing an interpreter, lexical scope is harder to implement than dynamic scope. Dynamic scope is 'natural' when writing an interpreter.

## Question 4

What is the output of the following code?

In [ ]:
for i in range(10):
    i = i + 1
print(i)

The output is 10, not an error message

When a 'new scope' starts in Python, it does use lexical scope, but when saving the environment, it saves a POINTER to the current environment, rather than a copy of the values in the environemt. 

----
# Closure Conversion

## Question 5

Transform the following program into an `Llambda` program by performing closure conversion.

```
z = 1
def f():
    z = 3
    def g():
        print(z)
    return g

z = 5
f()()
```

In [6]:
from typing import Tuple

# closure representation : 
#tuple with :
#- pointer to the function itself 
# - values for all function's free variables
# change functions so they take their own closures as an argument
# before doing anything else, read values of free variables from the closure argument
z = 1
def f_fun(closure):
    z = 3
    def g_fun(closure):
        z= closure[1]
        print(z)
    g = (g_fun,z)
    z= 50
    # z's value here is it's value at *function definition time*
    return g
f = (f_fun,) # f has no free variables

z = 5
tmp1 = f[0](f) # turn function calls f() into f[0](f, args....)
#tmp1()
tmp1[0](tmp1)

3


## Question 6

Describe the steps of closure conversion.

There are three major steps :
1. Modify the function definition to construct a closure representation
    - Our representation is a tuple
    - First value is a pointer to the function itself
    - Later values are the values of free variables from the function definition

2. Modify the function itself to take closure representaion as an argument and initialize closed-over variables to the values they have in closure representation.
    - Rename the function definition
    - change the arguments of the function to include closure representation
3. Modify the function calls of the form `f(args....)` to `f[0](f,args....)`

## Question 7

Describe the changes to the compiler required to implement closure conversion.

Add one new pass after the second typechecker called `convert_to_closures`.
- It has similar structure to RCO
- It performs the transformation from question 6
- In cc_stmt, for the function definition case, do the transformation described above 
    - Find the free variables of the function definiton
    - Modify the function def (1st stmt)
    - Construct the closure representation (2nd stmt)
    - one solution: modify global list of functions so that you know new name of the function is function
- In cc_exp, for the Call case, do the transformation described above
    - Modify all function calls to closure representation ( even if they have no free variables)

Other changes : 
- Implement lexical scope in typechecker(happens by default)
- Before : typechecker stores the names of all functions
- After : "functions" will be represented by closures

Select Instructions :

- Variable case : Assign(x,Var(y)) when y is a function
    - Before: turned into leaq instruction


----
# Dynamic Typing

In [ ]:
If 5==6:
    x = 3
else :
    x = True
# no static typing rules that reject programs
# no type annotations
def f(x):
    return x+1

## Question 8

Describe the challenge of compiling dynamically-typed programs into x86 assembly.

In our current compiler, we rely on types in important ways:
1. We disallow certain operations (eg: cant add bool to an int)
2. We produce different code depending on types

In a dynamically typed language:
1. We might not know a variables type in compile time
2. In our current compiler, no type information attached to values at runtime

## Question 9

Describe an approach for compiling a dynamically-typed language `Ldyn` to x86 assembly.

We are going to introduce a *gradually-typed* language called `Lany`. In a gradually typed language, annotations are optional and the `Any` type can represent any type in the language. Python is a gradually typed language(now).

Compiler `Ldyn` to `Lany`, the compile `Lany` to x86 Assembly code. The `Lany` language has the new type `Any` which represents any type.

1. The `Any` type is the type of a *tagged value* . The value itself will have a tag attached to it that says *at runtime* what the type of the value is.(eg: we can use the tag to differentiate bool and int)

2. We introduce `Inject` to convert a statically typed value into a value of type `Any`. When it does the conversion, it adds the tag indicating that the tagged value has the staticaly-known type of the input.

3. we introduce `project` to convert a value of type `Any` to a statically typed value with a desired type. When it does the conversion, `project` (1) checks at runtime that the tag on the input value matches the desired type and exits if not; (2) removes the tag

Process :
1. We compile `Ldyn` to `Lany` by adding inject and project operators.
2. For each constant: use inject to convert the constant into a tagged Any value
3. For each primitive: use project to project the inputs to the corrected expected types; use inject to convert the output to Any.

After we do this we can actually run the static typechecker(after adding `Any` as a type). Every variaable will have the type any, and the program will typecheck

## Question 10

Transform the following `Ldyn` program into an `Lany` program using `inject` and `project`.

```
x = 5
y = 6
z = x + y
```

In [10]:
from typing import TypeVar
T = TypeVar('T')

from dataclasses import dataclass
@dataclass
class AnyVal:
    val: any
    tag: type
        
def inject(val: T) -> AnyVal: # call this on something with a known static type to get Any
    return AnyVal(val, type(val))

def project(tagged_val: AnyVal, t: T) -> T: # call this on Any to get a desired type
    if tagged_val.tag == t:
        return tagged_val.val
    else:
        raise Exception('run-time type error!')

In [13]:
x = inject(5)
y = inject(6)
tmp1 = project(x, int)
tmp2 = project(y,int)
tmp3 = tmp1 + tmp2
x = inject(tmp3)
x

AnyVal(val=11, tag=<class 'int'>)

## Question 11

Describe the changes to the compiler for dynamic typing.

Add the Any type to the typechecker(add as an AnyT dataclass); add typing rules for inject and project

Two new passes : 
1. ` cast_insert` adds the above (inject and project). Put it before the typechecker(because you cant typecheck a program that is dynamically typed). The output of this pass is `Lany`.

2. `reveal_casts` compiles the casts into lower-level primitives. Put it after RCO because it introduces new control flow.

Structure of both passes is similar to RCO (follows grammar)

The reveal casts pass compiles both inject and project:
1. Project compiles into an if statement that checks if the tag is correct and returns the value if so; otherwise exits the program.
2. inject compiles into a `make_any` primitive that attaches a tag (select instructions will compile it further)

For part #1 use two primitives: `tag_of` and `value_of`
- tag_of returns a tag of a value of type`Any`
- `value_of ` returns the value of a value of type `Any`

For `x = project(y, int)` :
we produce :

```
if tag_of(y) ==1 :
    x = value_of(y)
else:
    exit()
```

can calculate the tag value at compile time ( use the tag values from the section 10.2 in the book; convert to decimal values).
We will deal the three remaining new primitives in select_instructions.


## Question 12

Write assembly language instructions for the following `Cany` statements. Reference Section 10.9 in the textbook.

1. `x = make_any(5, 1)`
2. `tmp_4 = tag_of(x)`
3. `tmp_1 = value_of(x)`

We'll compile these these into x86 Assembly instructions in select_instructions (one new case in si_stmnt per primitive)

1. Make_any adds the tag: shifts the value 3 bits to the left, and adds the tag to the value

` x = make_any(5,10)`
=>
```
salq $3, #x -- shifts left by 3 bits
orq $1, #x --- adds the tag 001
```
2. tag_of gets JUST the tag piece of a tagged value

`tmp_4 = tag_of(x)`
=>

```
movq #x, #tmp_4  -- copy the tagged value to the destination 
andq $7, #tmp_4  -- erase everything except the tag( 7 = 0b111)
```

3. Value_of gets JUST the value of the tagged value

`tmp_1 = value_of(x)` 
=>
movq #x, #tmp_1  -- move the tagged value to the destination 
sarq $3, #tmp_1  -- shift the tagged value 3 bits to the right, erasing the tag

For the exit primitive, you can jump to `main_conclusion`(exits the project)


## Question 13

What impact will these changes have on the performance of our compiled code? What would happen if we skipped this new code?

Think about a simple definition : `x= a + b`

In our statically-typed language, this turns into a single addq instruction 

In the dynamically-typed version, it turns into : 
- Atleast two if statements
- Atleaset three shifts

Each of these extra operations adds overhead. What was 1 instruction might be more like 10-20 instructions. This has a huge impact on performance, sometimes > 10x.

This is one reason dynamically-typed programming languages have a reputation for being slow(the other is that they're often implemented using interpreters)

This seems super bad!!! One implication is that you can view static typing as a performance optimization.

Options:
1. Skip checks. Outcome can be : operate on the byte-level representation of a value, while considering it to represent the wrong type. ( e.g  f+5 where f is a fucntion pointer) This is basically what C does.

2. Use a  Fancy compiler, like a Just-In-Time compiler (JIT). A JIT compiles code to native x86 Assembly *at runtime* . The reason is that at runtime, you know the values of all inputs to a piece of code- that means you also know the types. So you can produce code without run-time checks, even for a dynamically-typed language. 